In [1]:
import numpy as np
import joblib
from sklearn.metrics import r2_score ,mean_squared_error
from sklearn.linear_model import Lasso
from xgboost import XGBRegressor
import pandas as pd
from sklearn.svm import SVR

def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# data

In [2]:
# all data
data = joblib.load('../data/dataset.pkl')
print(data['action_col'])
b = data['stream']
batch_size = b.shape[0]
A = data['action']
A = A.reshape(len(b),-1)
print(A.shape,b.shape)

['MLPAP_FQ-0619.PV', 'MLPAP_TJC-0627.PV', 'MLPAP_TRC-0610.PV', 'MLPAP_FR-0632.PV', 'MLPAP_TJC-0757A.PV', 'MLPAP_PIC-0761.PV', 'MLPAP_TJC-1721.PV', 'MLPAP_TRC-1734.PV', 'MLPAP_TRC-0747.PV', 'MLPAP_TJ-0757B.PV', '濃度']
(126696, 396) (126696, 1)


In [3]:
# split data
sp1 = 0
sp2 = -1585
A_train = A[sp1:sp2]
b_train = b[sp1:sp2]
A_test = A[sp2:]
b_test = b[sp2:]

# Lasso find w

In [4]:
model = Lasso(0.0001)
model.fit(A_train,b_train)

C:\Users\skywalker0803r\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.6565142694156805, tolerance: 0.14950333468310498
  positive)


Lasso(alpha=0.0001, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

# evaluate

In [5]:
y_pred = model.predict(A_train)
y_real = b_train
score = pd.DataFrame(index = ['train'],columns = ['R2','MSE','MAPE'])
score['R2'] = r2_score(y_real,y_pred)
score['MSE'] = mean_squared_error(y_real,y_pred)
#score['MAPE'] = mape(y_real,y_pred)
score

,R2,MSE,MAPE
train,0.904118,0.001146,NaN


In [6]:
y_pred = model.predict(A_test)
y_real = b_test
score2 = pd.DataFrame(index = ['test'],columns = ['R2','MSE','MAPE'])
score2['R2'] = r2_score(y_real,y_pred)
score2['MSE'] = mean_squared_error(y_real,y_pred)
score2['MAPE'] = mape(y_real,y_pred)
score2

,R2,MSE,MAPE
test,-3.09626,0.000812,4.565275


In [7]:
score.append(score2)

,R2,MSE,MAPE
train,0.904118,0.001146,NaN
test,-3.096260,0.000812,4.565275


In [8]:
joblib.dump(model.coef_,'../model/stream_lasso_weight.pkl')

['../model/stream_lasso_weight.pkl']